# Trace Dataset Visualization (N=100)

Visualize the Azure trace dataset and ShareGPT prompts used by trace-sharegpt-hybrid.py

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from datetime import datetime

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Load Azure Trace (Timestamps)

In [ ]:
# Load Azure trace
azure_trace_file = '/home/skt6g/AI-RAN/KubeSMO/data/AzureLLMInferenceTrace_conv_1week.csv'
N = 100

# Read first N rows
df_azure = pd.read_csv(azure_trace_file, nrows=N)

print(f"📊 Loaded {len(df_azure)} Azure trace requests")
print(f"\nColumns: {df_azure.columns.tolist()}")
df_azure.head()

## 2. Parse Timestamps

In [ ]:
def parse_timestamp(ts_str):
    """Parse Azure timestamp"""
    ts_str = ts_str.strip()
    
    # Remove Z suffix
    if ts_str.endswith('Z'):
        ts_str = ts_str[:-1]
    
    # Remove timezone offset
    if '+' in ts_str:
        ts_str = ts_str.split('+')[0]
    elif ts_str.count('-') > 2:
        parts = ts_str.rsplit('-', 1)
        if ':' in parts[1]:
            ts_str = parts[0]
    
    # Handle fractional seconds
    if '.' in ts_str:
        parts = ts_str.split('.')
        fractional = parts[1][:6].ljust(6, '0')
        ts_str = f"{parts[0]}.{fractional}+00:00"
    else:
        ts_str = f"{ts_str}+00:00"
    
    return datetime.fromisoformat(ts_str)

# Parse timestamps
df_azure['timestamp'] = df_azure['TIMESTAMP'].apply(parse_timestamp)

# Calculate inter-arrival times
df_azure['inter_arrival_sec'] = df_azure['timestamp'].diff().dt.total_seconds()

# Calculate time from start
start_time = df_azure['timestamp'].iloc[0]
df_azure['time_from_start_sec'] = (df_azure['timestamp'] - start_time).dt.total_seconds()

print(f"\n⏱️  Trace Duration: {df_azure['time_from_start_sec'].max():.1f} seconds")
print(f"   ({df_azure['time_from_start_sec'].max()/60:.2f} minutes)")
print(f"\n📈 Inter-arrival Time Statistics:")
print(f"   Mean: {df_azure['inter_arrival_sec'].mean():.2f} sec")
print(f"   Min:  {df_azure['inter_arrival_sec'].min():.2f} sec")
print(f"   Max:  {df_azure['inter_arrival_sec'].max():.2f} sec")
print(f"   p50:  {df_azure['inter_arrival_sec'].median():.2f} sec")
print(f"   p95:  {df_azure['inter_arrival_sec'].quantile(0.95):.2f} sec")

## 3. Load ShareGPT Prompts

In [ ]:
# Load ShareGPT
sharegpt_file = '/home/skt6g/AI-RAN/KubeSMO/data/sg_90k_part1.json'

with open(sharegpt_file, 'r', encoding='utf-8') as f:
    sharegpt_data = json.load(f)

# Extract prompts
prompts = []
for item in sharegpt_data:
    conv = item.get('conversations', [])
    for msg in conv:
        if msg.get('from') == 'human':
            prompt = msg.get('value', '').strip()
            if prompt and len(prompt) > 10:
                prompts.append(prompt)

print(f"\n📝 Loaded {len(prompts)} ShareGPT prompts")

# Sample the first N prompts (cycling behavior)
selected_prompts = [prompts[i % len(prompts)] for i in range(N)]
prompt_lengths = [len(p) for p in selected_prompts]

print(f"\n📏 Prompt Length Statistics (first {N} used):")
print(f"   Mean: {np.mean(prompt_lengths):.1f} chars")
print(f"   Min:  {np.min(prompt_lengths)} chars")
print(f"   Max:  {np.max(prompt_lengths)} chars")
print(f"   p50:  {np.median(prompt_lengths):.1f} chars")
print(f"   p95:  {np.percentile(prompt_lengths, 95):.1f} chars")

## 4. Visualize Request Arrival Pattern

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Request arrivals over time
axes[0, 0].scatter(df_azure['time_from_start_sec'], 
                   range(len(df_azure)), 
                   alpha=0.6, s=30)
axes[0, 0].set_xlabel('Time from Start (seconds)', fontsize=12)
axes[0, 0].set_ylabel('Request Number', fontsize=12)
axes[0, 0].set_title(f'Request Arrival Pattern (N={N})', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# 2. Inter-arrival time distribution
axes[0, 1].hist(df_azure['inter_arrival_sec'].dropna(), bins=30, alpha=0.7, edgecolor='black')
axes[0, 1].axvline(df_azure['inter_arrival_sec'].mean(), 
                   color='red', linestyle='--', linewidth=2, label=f'Mean: {df_azure["inter_arrival_sec"].mean():.2f}s')
axes[0, 1].set_xlabel('Inter-arrival Time (seconds)', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title('Inter-arrival Time Distribution', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# 3. Cumulative requests
axes[1, 0].plot(df_azure['time_from_start_sec'], range(1, len(df_azure)+1), linewidth=2)
axes[1, 0].set_xlabel('Time from Start (seconds)', fontsize=12)
axes[1, 0].set_ylabel('Cumulative Requests', fontsize=12)
axes[1, 0].set_title('Cumulative Request Arrivals', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Prompt length distribution
axes[1, 1].hist(prompt_lengths, bins=30, alpha=0.7, edgecolor='black', color='green')
axes[1, 1].axvline(np.mean(prompt_lengths), 
                   color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(prompt_lengths):.1f} chars')
axes[1, 1].set_xlabel('Prompt Length (characters)', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title('ShareGPT Prompt Length Distribution', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('trace_dataset_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💾 Saved visualization to: trace_dataset_visualization.png")

## 5. Request Rate Analysis

In [ ]:
# Calculate request rate in time windows
window_size = 10  # seconds
max_time = df_azure['time_from_start_sec'].max()
time_bins = np.arange(0, max_time + window_size, window_size)

request_counts = []
for i in range(len(time_bins) - 1):
    count = ((df_azure['time_from_start_sec'] >= time_bins[i]) & 
             (df_azure['time_from_start_sec'] < time_bins[i+1])).sum()
    request_counts.append(count)

request_rates = [count / window_size for count in request_counts]

plt.figure(figsize=(14, 5))
plt.bar(time_bins[:-1], request_rates, width=window_size*0.9, alpha=0.7, edgecolor='black')
plt.axhline(np.mean(request_rates), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {np.mean(request_rates):.2f} req/s')
plt.xlabel('Time from Start (seconds)', fontsize=12)
plt.ylabel('Request Rate (requests/second)', fontsize=12)
plt.title(f'Request Rate Over Time (Window={window_size}s)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('request_rate_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Request Rate Statistics ({window_size}s windows):")
print(f"   Mean:  {np.mean(request_rates):.2f} req/s")
print(f"   Min:   {np.min(request_rates):.2f} req/s")
print(f"   Max:   {np.max(request_rates):.2f} req/s")
print(f"   p50:   {np.median(request_rates):.2f} req/s")
print(f"   p95:   {np.percentile(request_rates, 95):.2f} req/s")
print(f"\n💾 Saved visualization to: request_rate_analysis.png")

## 6. Sample Prompts Preview

In [ ]:
print("\n📝 Sample Prompts (first 5):\n")
print("="*80)
for i, prompt in enumerate(selected_prompts[:5]):
    print(f"\nPrompt {i+1} ({len(prompt)} chars):")
    print("-"*80)
    # Show first 200 chars
    preview = prompt[:200] + "..." if len(prompt) > 200 else prompt
    print(preview)
    print("="*80)

## 7. Summary Statistics Table

In [ ]:
summary_df = pd.DataFrame({
    'Metric': [
        'Total Requests',
        'Trace Duration (sec)',
        'Trace Duration (min)',
        'Mean Inter-arrival (sec)',
        'Mean Request Rate (req/s)',
        'Mean Prompt Length (chars)',
        'Max Prompt Length (chars)',
        'Total ShareGPT Prompts Available'
    ],
    'Value': [
        N,
        f"{df_azure['time_from_start_sec'].max():.1f}",
        f"{df_azure['time_from_start_sec'].max()/60:.2f}",
        f"{df_azure['inter_arrival_sec'].mean():.2f}",
        f"{np.mean(request_rates):.2f}",
        f"{np.mean(prompt_lengths):.1f}",
        f"{np.max(prompt_lengths)}",
        len(prompts)
    ]
})

print("\n📊 Dataset Summary:")
print("="*80)
print(summary_df.to_string(index=False))
print("="*80)

## 8. Export Data for Analysis

In [ ]:
# Create analysis dataframe
analysis_df = pd.DataFrame({
    'request_id': range(N),
    'timestamp': df_azure['timestamp'],
    'time_from_start_sec': df_azure['time_from_start_sec'],
    'inter_arrival_sec': df_azure['inter_arrival_sec'],
    'prompt_length_chars': prompt_lengths,
    'prompt_preview': [p[:100] for p in selected_prompts]  # First 100 chars
})

# Save to CSV
output_file = 'trace_dataset_analysis.csv'
analysis_df.to_csv(output_file, index=False)
print(f"\n💾 Saved analysis data to: {output_file}")

analysis_df.head(10)